In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" #use this accordign to your GPU setup
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import  LoraConfig, get_peft_model, PeftModel


In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Use teh base model whihc you trained below, can be llama, mixtral etc
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)
print(model)

 # path to the checkpoint folder, check your path as the checkpoint number might be different
lora =  "./mistral-lora-instruct-shapeskeare/checkpoint-32/"

model = PeftModel.from_pretrained(model, lora)



In [ ]:
def generate_prompt(user_query):  #The prompt format is taken from the official Mistral huggingface page, yformat for each model might differ
    return "[INST]" + user_query + "[/INST]" 

def evaluate(question):

    prompt= generate_prompt(question)
    inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    with torch.no_grad():
        gen_tokens = model.generate(
             input_ids=input_ids,
            attention_mask=inputs["attention_mask"].cuda(),
           max_new_tokens=512,
            do_sample=True,
            temperature=0.8,
            top_p=0.95,
            eos_token_id=2,
                )

    out=tokenizer.decode(gen_tokens[0],
        skip_special_tokens=False)

    return out



In [ ]:
questions_eval = [
    "What is your favorite color?",
    "How many continents are there in the world?",
    "What year was the first iPhone released?",
    "What is the capital of France?",
    "Who wrote 'To Kill a Mockingbird'?",
    "What gas do plants breathe in that humans and animals breathe out?",
    "How many planets are in our solar system?",
    "What is the boiling point of water?",
    "What is the largest ocean on Earth?",
    "Who is the current president of the United States?"
]


for question in  questions_eval:
    a=evaluate(question)
    print(a)
    print()
    print("-"*50)